# Data Mining Projekt
## Wie soll der nächste Charakter aussehen?
___

## Problemdefiniton
In League of Legends beeinflussen neue Champions die Spielbalance. Doch sind bestimmte Champion-Typen unterrepräsentiert? Diese Arbeit nutzt Machine Learning, um vorherzusagen, welche Champion-Eigenschaften im aktuellen Spielgewicht fehlen.

So wird sich die zentrale Frage gestellt: Welche Klassen-Rollen-Kombination ist bei sonst durchschnittlichen Werten im derzeitigen Spielökosystem am stärksten unterrepräsentiert und könnte als Grundlage für die Entwicklung eines neuen Champions dienen?

## Import - Datenauswahl

### Import notwendiger Bibliotheken


In [ ]:
# Datenverarbeitung & Numerik
import pandas as pd  
import numpy as np  

# Visualisierung
import matplotlib.pyplot as plt  
import seaborn as sns  
%matplotlib inline  

# Datenaufbereitung & Preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder  
from sklearn.model_selection import train_test_split  
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Machine Learning Modelle
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor  
from sklearn.linear_model import LogisticRegression, LinearRegression  
from sklearn.svm import SVC  
from sklearn.neural_network import MLPRegressor  
from xgboost import XGBClassifier, XGBRegressor 
from xgboost import plot_importance


# Modellbewertung
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  
from sklearn.metrics import r2_score, mean_squared_error 
from sklearn.model_selection import cross_val_score

# Hyperparameter-Tuning
from sklearn.model_selection import GridSearchCV

# Explorative Datenanalyse
#from ydata_profiling import ProfileReport  

# Sonstiges
import itertools  
from itertools import product
import pickle  

### Datenimport

Der folgende Datensatz wurde auf der Website "Kaggle" gefunden, hat seinen Ursprung jedoch in
der Analyse- und Datenplattform MetaSRC. Diese Plattform sammelt, aggregiert und analysiert Match-Daten
um die Trends des Spiels League of Legends darzustellen.
Die Daten werden direkt aus öffentlichen Riot Games-APIs bezogen, wodurch die Statistiken
patch-basiert aktualisiert werden.

In [ ]:
# Datensatz laden
df = pd.read_csv('../Daten/League of Legends Champion Stats 12.1.csv', sep=';')
df.head()

Der nächste Datensatz entstammt auch Kaggle, hat aber seinen Ursprung in dem "League of Legends Wiki Champion Data Module".
Da dieser Veröffentlichungsdaten beinhaltet, eignet sich dieser für Zeitreihenanalysen
Der Fokus liegt allerdings auf ersterem Datensatz, dieser dient nur als Hilfe für die Zeitreihenanalyse


In [ ]:
df_basic = pd.read_csv('../Daten/200125_LoL_champion_data.csv')
df_basic.head()

### Merging

In [ ]:
# Verbunden werden die Datensätze durch den gemeinsamen Schlüssel des Champion-Namens
# Dabei wollen wir zum ersten Datensatz die Informationen bezüglich Veröffenlichungsdatum hinzufügen

# Spaltennamen für Konsistenz anpassen
df_basic.rename(columns={'apiname': 'Name'}, inplace=True)

# 'date' Spalte in datetime umwandeln
df_basic['date'] = pd.to_datetime(df_basic['date'], errors='coerce')

# Füge die Veröffentlichungsdaten aus df_basic zum Haupt-Datensatz df hinzu
df = pd.merge(df, df_basic[['Name', 'date']], on='Name', how='left')

df.head()

## EDA und Preprocessing

Da dies ein Machine Learning Projekt ist, ist es wichtig, dass wir in den folgenden Abschnitten einen Überblick über die Daten erhalten und die Daten entsprechend vorbereiten.
Für die spätere Modellerstellung ist es dabei wichtig, dass Nullwerte entfernt werden, die Daten in numerische Werte umgewandelt werden und die Daten in Trainings- und Testdaten aufgeteilt werden.
Auch Skalierung der Daten ist wichtig, um sicherzustellen, dass die Modelle korrekt trainiert werden.
Der erste Schritt ist es, sich die Daten genauer anzuschauen, um zu sehen, welche Daten vorhanden sind und wie sie aussehen.

In [ ]:
df.shape

In [ ]:
# Übersicht über die Daten
df.info()

In [ ]:
# Beschreibung der numerischen Spalten
print(df.describe())

### Zeitreihenanalyse

In [ ]:
# Veröffentlichungsdatum extrahieren
df['release_year'] = df['date'].dt.year

# Date spalte entfernen
df.drop('date', axis=1, inplace=True)

In [ ]:
# Anzahl der veröffentlichten Champions pro Jahr berechnen
release_counts = df['release_year'].value_counts().sort_index()

In [ ]:
# Veröffentlichung pro Rolle und Klasse
role_counts = df.groupby(['release_year', 'Role']).size().unstack()
class_counts = df.groupby(['release_year', 'Class']).size().unstack()

In [ ]:
# Anzahl veröffentlichter Champions pro Jahr 
plt.figure(figsize=(12, 6))
plt.bar(release_counts.index, release_counts.values, color='skyblue', edgecolor='black')
plt.xlabel("")
plt.ylabel("")
plt.title("Anzahl der veröffentlichten Champions pro Jahr")
plt.xticks(release_counts.index) 
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

### Preprocessing: In numerische Werte umwandeln

In [ ]:
# Umwandlung von Prozentangaben in numerische Werte
df["Win %"] = df["Win %"].str.replace('%', '').astype(float)
df["Role %"] = df["Role %"].str.replace('%', '').astype(float)
df["Pick %"] = df["Pick %"].str.replace('%', '').astype(float)
df["Ban %"] = df["Ban %"].str.replace('%', '').astype(float)


print(df["Pick %"].describe())


### Betrachtung fehlender Werte

In [ ]:
# Fehlende Werte
print(df.isnull().sum())

In [ ]:
# Alle Zeilen ausgeben, in denen das 'release_year' fehlt
missing_year_rows = df[df["release_year"].isnull()]
print(missing_year_rows)


In [ ]:
mean_year = round(df["release_year"].mean())  # Aufrunden, da Jahre ganzzahlig sind
df["release_year"].fillna(mean_year, inplace=True)


In [ ]:
print(df.isnull().sum())

In [ ]:
# Die Zeile ausgeben, in welcher ein Wert für 'Class' fehlt
missing_names_rows = df[df["Class"].isnull()]  
print(missing_names_rows)

In [ ]:
# Den Champion 'Lillia' anzeigen
df[df['Name'] == 'Lillia']

In [ ]:
# Da ich weiss, dass Lillia Fighter ist, kann ich den Wert direkt einfügen
df.loc[df['Name'] == 'Lillia', 'Class'] = 'Fighter'

In [ ]:
print(df.isnull().sum())

### Weitere EDA-Visualisierungen

In [ ]:
# Veröffentlichungen nach Rolle
plt.figure(figsize=(12, 6))
role_counts.plot(kind='line', marker='o', linestyle='-', figsize=(12, 6))
plt.xlabel("Jahr")
plt.ylabel("")
plt.title("Champion-Veröffentlichungen pro Rolle")
plt.xticks(release_counts.index)
plt.legend(title="Rolle")
plt.grid(True)
plt.show()

In [ ]:
# Veröffentlichungen nach Klasse
plt.figure(figsize=(12, 6))
class_counts.plot(kind='line', marker='s', linestyle='-', figsize=(12, 6))
plt.xlabel("Jahr")
plt.ylabel("")
plt.title("Champion-Veröffentlichungen pro Klasse")
plt.xticks(release_counts.index)
plt.legend(title="Klasse")
plt.grid(True)
plt.show()

In [ ]:
# Pairplot zur Visualisierung der Feature-Zusammenhänge
numerical_features = df.select_dtypes(include=[np.number]).columns
sns.pairplot(df[numerical_features])

In [ ]:
highlight = {"Tank", "Mage", "Fighter"}
purple = "#800080"   

order = ["Fighter", "Mage", "Assassin", "Marksman", "Tank", "Support"]

palette = {c: ("grey" if c not in highlight else purple) for c in order}

plt.figure(figsize=(10, 5))
ax = sns.boxplot(
    x="Class",
    y="Pick %",
    data=df,
    order=order,
    palette=palette
)

ax.set_xlabel("")
ax.set_ylabel("")
ax.set_title("Pickrate pro Champion-Klasse")
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Histogramm der Winrate

plt.figure(figsize=(8,5))
df['Win %'].hist(bins=20, color='grey', edgecolor='black')
plt.grid(False)
plt.title('Verteilung der Winrate der Champions')
plt.xlabel('Winrate')
plt.ylabel('Anzahl Champions')
plt.show()

### Ydata Profiling Report erstellen

In [ ]:
# Bevor die Namensspalte entfernt wird, speichern wir den df in einer neuen Variable
df_name = df.copy()

In [ ]:
df.head()

In [ ]:
df_name.head()

In [ ]:
# Für den ydata_profiling Report entfernen wir nicht float Datentypen wie 'Name'
df_profile_report = df.drop(['Name'], axis=1)
df_profile_report = df_profile_report.drop(['Class'], axis=1)
df_profile_report = df_profile_report.drop(['Role'], axis=1)
df_profile_report = df_profile_report.drop(['release_year'], axis=1)
df_profile_report = df_profile_report.drop(['Tier'], axis=1)

# Ydata Profiling Report erstellen
#profile = ProfileReport(df_profile_report, explorative=True)

# den Profile Report als HTML-Datei speichern
#ytrain = False
#if ytrain:
    #profile.to_file("ydata_profiling_report.html")

## Modellauswahl

Unser Modell soll vorhersagen, ob und wie Championeigenschaften wie deren Rolle zu einer Unterrepräsentation beitragen. Dazu werden mehrere Machnine Learning Modelle trainiert und anschließend verglichen. Die Modelle sind: Random Forest, Logistic Regression, Support Vector Machine und XGBoost.

### Feature Auswahl und Test-Train-Split

Zuerst wird die Zielvariable (Pick %) von den Features getrennt, und die Daten werden im Verhältnis 80:20 in Trainings- und Testsets aufgeteilt. Zudem wird die Spalte "Name" entfernt, da sie für das Modell nicht einfach in Kategorien oder numerische Werte umgewandelt werden kann. Auch die Variable "Release Year" wird entfernt, da diese keinen logischen Zusammenhang mit der Pickrate hat.

In [ ]:
# Die Spalte 'name' wird entfernt
df.drop(columns=["Name"], inplace=True, errors="ignore")

In [ ]:
# Zielvariable definieren
X = df.drop(columns=["Pick %"]) 
y = df["Pick %"]

In [ ]:
# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

### Preprocessing 2

Hier kombiniert eine Pipeline das Preprocessing.

In [ ]:
# Identifikation numerischer und kategorischer Features
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object"]).columns.tolist()

# Preprocessing Pipeline mit OrdinalEncoder (numerische Kodierung von Kategorien)
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), categorical_features)  
])


### Modelltestung

Nun werden die Modelle getestet, um zu sehen, welches Modell am besten performt.

In [ ]:
# Modelle
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=100, random_state=42),
    "Neural Network": MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
}

In [ ]:
# Training und Evaluation der Modelle
results = {}

for name, model in models.items():
    print(f"Training {name}...")
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("model", model)
    ])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    results[name] = {"R² Score": r2, "RMSE": rmse}
    print(f"{name}: R² = {r2:.4f}, RMSE = {rmse:.4f}\n")

In [ ]:
# Bestes Modell basierend auf dem höchsten R² Score ermitteln
best_model_name = max(results, key=lambda k: results[k]["R² Score"])
print(f"Bestes Modell: {best_model_name}")

# Bestes Modell aus dem Dictionary holen
best_model = models[best_model_name]


In [ ]:
# Visualisierung der Vorhersage
plt.figure(figsize=(10, 5))
plt.scatter(y_test, y_pred, color='purple', label="Vorhersagen")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], linestyle='--', color='black', label="Ideal")
plt.xlabel('')
plt.ylabel('')
plt.legend()
plt.show()


In [ ]:
# Residuen berechnen
residuals = y_test - y_pred

# Residuenplot
plt.figure(figsize=(10, 5))
plt.scatter(y_pred, residuals, alpha=0.7, color="darkred", edgecolor="black")
plt.axhline(y=0, linestyle='--', color='black')
plt.title("Residuenanalyse")
plt.xlabel("Vorhergesagte Pickrate")
plt.ylabel("")
plt.grid(True)
plt.show()


## Training des Modells


### Hyperparameter-Tuning mit GridSearchCV


In [ ]:

# Gitter für die Hyperparameter
param_grids = {
    "Random Forest": {
        "model__n_estimators": [100, 200],
        "model__max_depth": [None, 10, 20]
    },
    "XGBoost": {
        "model__n_estimators": [100, 200],
        "model__learning_rate": [0.01, 0.1, 0.2],
        "model__max_depth": [3, 5]
    },
    "Neural Network": {
        "model__hidden_layer_sizes": [(64, 32), (128, 64)],
        "model__alpha": [0.0001, 0.001]
    }
}

results = {}

for name, model in models.items():
    print(f"Training {name}...")
    
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("model", model)
    ])
    
    if name in param_grids:
        grid_search = GridSearchCV(pipeline, param_grids[name], cv=3, scoring='r2', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        print(f"Beste Parameter für {name}: {grid_search.best_params_}")
    else:
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        best_model = pipeline
    
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    results[name] = {"R² Score": r2, "RMSE": rmse}
    print(f"{name}: R² = {r2:.4f}, RMSE = {rmse:.4f}\n")


In [ ]:
# Verwende bestes Modell aus GridSearch falls vorhanden
best_xgb = XGBRegressor(
    n_estimators=200,      
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", best_xgb)
])

# Cross-Validation auf Trainingsdaten 
cv_scores = cross_val_score(
    pipeline,
    X_train,
    y_train,
    cv=5,
    scoring="r2"
)

print("CV R² Scores:", cv_scores)
print("Mean CV R²:", cv_scores.mean())
print("Std CV R²:", cv_scores.std())

# 2) Final Fit + Evaluation auf Testset
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Final Test R²: {r2:.4f}")
print(f"Final Test RMSE: {rmse:.4f}")

# Modell speichern
with open("lol_pickrate_model.pkl", "wb") as file:
    pickle.dump(pipeline, file)

### Feature Importance analysieren

In [ ]:
xgb_model = pipeline.named_steps["model"]

# Plotten
importances = xgb_model.feature_importances_

# Feature-Namen aus dem Preprocessor holen
num_names = numeric_features
cat_names = categorical_features

feature_names = num_names + cat_names

xgb_model = pipeline.named_steps["model"]
importances = xgb_model.feature_importances_

for name, score in sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True):
    print(f"{name}: {score:.4f}")


## Championgenerierung / Ergebnisinterpretation

Jetzt wollen wir das beste Modell anwenden, um zu testen, ob es genau ist
Daher nehmen wir die Werte eine Champions der Reihe 1 und testen, ob das Modell 
die Pickrate korrekt vorhersagt
Dazu müssen wir die Werte nicht aus den Trainingsdaten nemhen, sondern aus den ursprünglichen Daten

In [ ]:
# Wir bedienen uns der vor langer Zeit erstellten "Backup" - Variable df_name,
# um die Spalte 'Name' wieder hinzuzufügen

df_name.head()

In [ ]:
# die echten Werte des Champions "Aatrox' anzeigen
champion = df_name[df_name['Name'] == 'Aatrox'].iloc[0].to_dict()
champion

In [ ]:
# Pickrate vorhersagen
with open("lol_pickrate_model.pkl", "rb") as file:
    model = pickle.load(file)

In [ ]:
# Werte des Champions in ein DataFrame umwandeln
champion_df = pd.DataFrame([champion])

# Namensspalte rausnehmen
champion_df = champion_df.drop(columns=["Name"], errors="ignore")
                               
# Pickrate vorhersagen
pickrate = model.predict(champion_df)[0]
print(f"Vorhergesagte Pickrate für Aatrox: {pickrate:.4f}%")

In [ ]:
# Werte des Champions "Xerath' betrachten
champion = df_name[df_name['Name'] == 'Xerath'].iloc[0].to_dict()
champion

In [ ]:
champion = df_name[df_name['Name'] == 'Xerath'].iloc[0].to_dict()
champion_df = pd.DataFrame([champion])
champion_df = champion_df.drop(columns=["Name"], errors="ignore")
pickrate = pipeline.predict(champion_df)[0]
print(f"Vorhergesagte Pickrate für Xerath: {pickrate:.4f}%")

Die Pickrates wurden super vorhergesagt! :D

In [ ]:
# Champion mit niedrigster Pickrate als Vergleich anzeigen
champion_underrep = df_name.loc[df_name["Pick %"].idxmin()]
print("Champion mit den am niedrigsten Pickrate:")
print(champion_underrep)

Jetzt werden durchschnittliche Werte für alle Variablen außer Class und Role genommen, um diese kategorialen Variablen vorherzusagen. Es soll nämlich erkannt werden, welche Klasse und Rolle bei durchschnittlichen Werten am unterrepräsentiertesten ist. 

In [ ]:
# Modell laden
with open("lol_pickrate_model.pkl", "rb") as file:
    model = pickle.load(file)

In [ ]:
# Definieren kategorialer Features
variable_features = ['Class', 'Role']

In [ ]:
# Erstellen aller möglichen Kombinationen aus Class und Role
combinations = list(product(
    df['Class'].dropna().unique(),
    df['Role'].dropna().unique()
))

In [ ]:
# Setzen aller anderen Features auf den Mittelwert
fixed_values = {}

for col in df.columns:
    if col not in variable_features + ['Name', 'Pick %']:
        if pd.api.types.is_numeric_dtype(df[col]):
            fixed_values[col] = df[col].mean()
        else:
            fixed_values[col] = df[col].mode()[0] 

In [ ]:
# Iterieren durch alle Kombinationen der kategorialen Werte
min_pred = float('inf')
best_candidate = None

for class_val, role_val in combinations:
    candidate = fixed_values.copy()
    candidate['Class'] = class_val
    candidate['Role'] = role_val

    candidate_df = pd.DataFrame([candidate])
    pred = model.predict(candidate_df)[0]

    if pred < min_pred:
        min_pred = pred
        best_candidate = candidate

print("Theoretische Champion-Komposition (bei durchschnittlicher Leistung):")
print(best_candidate)
print("Vorhergesagte Pickrate: {:.2f}%".format(min_pred))

## Ergebnisinterpretation

Die Analyse zeigt, dass die Rolle "Top" und die Klasse "Tank" (Class Tank) bei durschnittlichen Werten unterrepräsentierte Champions ausmacht. Besonders die Klasse lässt sich logisch nachvollziehen, beispielsweise wurden Champions der Klasse "Tank", in den letzten Jahren oft gar nicht oder selten pro Jahr veröffentlicht. Weiters wird auf diese Thematik in der schriftlichen Arbeit genauer eingegangen.